In [7]:
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

#
import matplotlib
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal


# 
import numpy as np
import os
from tqdm import trange
import parmap
import glob

# 
from scipy.io import loadmat
import scipy

# 
from Convert import Convert

Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# # data = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/post_processing/centres/2020-3-16_01_54_23_358257_compressed.avi.predictions.analysis_traces_reassembled_centres.npy')
# # print (data.shape)
# data = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed_fixed.npy')
# print (data.shape)

In [9]:
# 
fnames = [
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-6_0day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-8_0day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-9_1day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-11_day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-12_day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-13_day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-14_day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-15_day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-16_day_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-10_daytime_5mins_compressedTalmo_fixed.npy',
'/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-7_daytime_0_300sec_compressedTalmo_fixed.npy'
]

root_dir = '/media/cat/4TBSSD/dan/cohort1/compressed_slp/'
fnames = glob.glob(root_dir+"*_compressed_fixed.npy")

def run_trx_parallel(fname):
     #
    convert = Convert(fname)

    # 
    # convert.sexes = ['F','M','M','M']
    convert.sexes = [0,1,1,1]
    convert.fps = 25
    convert.start = 0 #int(0*60*convert.fps)
    convert.end = None #None #int(1*60*convert.fps)

    # start = 0
    # end = None
    
    convert.apply_median_filter = True
    convert.dtype = 'double'
    convert.animal_ids = np.arange(4)
    convert.scale = 1
    convert.make_movie = False
    #convert.axes_scale = 4

    # 
    convert.extension = '.avi'
    convert.convert_npy_to_jaaba()

if True:
    parmap.map(run_trx_parallel,
              fnames,
              pm_processes=16,
              pm_pbar=True)
else:
    
    for fname in fnames:
        run_trx_parallel(fnam)
    #


  0%|          | 0/470 [00:00<?, ?it/s]

 full features track data:  (89989, 4, 14, 2)


  9%|▉         | 7941/89989 [00:04<00:40, 2011.74it/s]

 full features track data:  (89988, 4, 14, 2)


  0%|          | 0/89988 [00:00<?, ?it/s]

...

 full features track data:  (89989, 4, 14, 2)

 97%|█████████▋| 87313/89988 [00:36<00:01, 2101.12it/s]

100%|█████████▉| 89546/89988 [00:12<00:00, 2091.00it/s]

 full features track data:  (89988, 4, 14, 2)


100%|██████████| 89988/89988 [00:24<00:00, 3626.49it/s]
472it [1:24:23, 10.73s/it]                         


In [25]:
fname = '/media/cat/4TBSSD/dan/cohort1/jaaba_test/movie1/trx.mat'

data = loadmat(fname)
print (len(data))
print (len(data['trx'][0][0]))
print (data['trx'][0][0])
theta = data['trx'][0][2][2].squeeze()
print (theta.shape)
plt.plot(theta)
plt.show()

4
19
(array([[796.33959961, 799.7789917 , 799.82263184, ..., 784.55865479,
        815.82635498, 812.80975342]]), array([[271.59109497, 271.86141968, 271.94561768, ..., 284.5559082 ,
        277.05532837, 277.10379028]]), array([[-0.05180243, -0.19633077, -0.1984491 , ..., -0.65860766,
        -0.506515  , -0.52507293]], dtype=float32), array([[103.545685, 103.53456 , 103.32587 , ..., 104.99651 ,  88.57386 ,
         85.40443 ]], dtype=float32), array([[43.991497, 36.096703, 36.092815, ..., 79.368355, 18.490385,
        18.395884]], dtype=float32), array([[7500]], dtype=uint16), array([[1]], dtype=uint8), array([[7500]], dtype=uint16), array([[0]]), array([[1]]), array([[398., 399., 399., ..., 392., 407., 406.]]), array([[135., 135., 135., ..., 142., 138., 138.]]), array([[-0.05180243, -0.19633077, -0.1984491 , ..., -0.65860766,
        -0.506515  , -0.52507293]], dtype=float32), array([[51., 51., 51., ..., 52., 44., 42.]], dtype=float32), array([[21., 18., 18., ..., 39.,  9.,  9.]], d

In [152]:
# 
from tqdm import tqdm
import warnings

from math import *
PI = 3.1415926535

def angle_trunc(a):
    while a < 0.0:
        a += pi * 2
    return a

def getAngleBetweenPoints(x_orig, y_orig, x_landmark, y_landmark):
    deltaY = y_landmark - y_orig
    deltaX = x_landmark - x_orig
    return angle_trunc(atan2(deltaY, deltaX)) #* 180 / PI


# 
tracks = np.load('/media/cat/4TBSSD/dan/cohort1/cohort1_training_daytime_processed/2020-3-6_0day_5mins_compressedTalmo_fixed.npy')
print (tracks.shape)

# 
start =0
end = 1000

# 
angles = np.zeros((end-start,
                        tracks.shape[1]),
                        'float32')+np.nan
axes = np.zeros((end-start,
                          tracks.shape[1],
                          2),
                            'float32')+np.nan
intercepts = np.zeros((end-start,
                        tracks.shape[1]),
                        'float32')+np.nan

#
deg_scale = 180/3.1415926
#deg_scale = 1
        
#      
for a in range(tracks.shape[1]):
    for k in tqdm(range(start, end,1)):
        #x = self.tracks[k,a,:,0]
        #y = self.tracks[k,a,:,1]
        x = tracks[k,a,5:10,0]
        y = tracks[k,a,5:10,1]
        idx = np.where(np.isnan(x)==False)[0]
        if idx.shape[0]>0:
            x=x[idx]
            y=y[idx]
            
            if x.shape[0]>=2:
                angle = getAngleBetweenPoints(x[0], y[0], 
                                              x[-1], y[-1])
            else:
                continue
            
            angles[k,a] = angle
            #intercepts[k,a] = 
            # stack locations
            locs = np.vstack((x,y)).T

            # rotate
            theta = np.radians(angle)
            c, s = np.cos(theta), np.sin(theta)
            R = np.array(((c, -s), (s, c)))
            locs_r = locs@R

            
#             # Reject outliers that are substantially outside of data
#             x = reject_outliers(locs_r[:,0])
#             y = reject_outliers(locs_r[:,1])

#             axes[k,a,0] = np.max(x)-np.min(x)
#             axes[k,a,1] = np.max(y)-np.min(y)


#             self.angles[k,a] = np.arctan(m)*deg_scale
    while True:
        idx = np.where(np.isnan(angles[:,a]))[0]
        if idx.shape[0]==0:
            break
        angles[idx,a] = angles[idx-1,a]
    
    angles[:,a] = scipy.ndimage.median_filter(angles[:,a], size=25)


100%|██████████| 1000/1000 [00:00<00:00, 24303.96it/s]

(7500, 4, 14, 2)


In [153]:
fig=plt.figure()
plt.plot(angles[:,0]*180/pi)
plt.show()

In [112]:
idx = np.random.choice(np.arange(1000), 10)
ctr=0
import math
for k in idx:
    ax=plt.subplot(2,5,ctr+1)

    for a in range(4):
        x = tracks[k,a,5:10,0]
        y = tracks[k,a,5:10,1]
        idx = np.where(np.isnan(x)==False)[0]
        if idx.shape[0]>0:
            x=x[idx]
            y=y[idx]
            
            #x = x-x[0]
            
            intercept = 0
            #print (math.radians(angles[k,a])*x,angles[k,a]*x )
            fit_eq = math.radians(angles[k,a])*x + y #-y[-1]
            
            
            #Plotting the data
            ax.plot(x, fit_eq, color = 'r', alpha = 0.5, label = 'Linear fit')

            plt.scatter(x,y)
    
    ctr+=1
plt.show()

In [ ]:

# get length
temp = data['allScores'][0][0][0][0][0].squeeze()
length = temp.shape[0]

# get starts/ends
t0s = data['allScores'][0][0][5].squeeze()
#print (t0s.shape)
t1s = data['allScores'][0][0][6].squeeze()

img = np.zeros((4,length),'float32')
img1 = np.zeros((4,length),'float32')
for k in trange(t0s.shape[0]):
#for k in [0]:
    starts = t0s[k].squeeze()
    ends = t1s[k].squeeze()
    #print (k,starts.shape)
    for p in range(starts.shape[0]):
        img[k,starts[p]:ends[p]]+=k+1
        img1[k,starts[p]:ends[p]]+=1
            

In [13]:
from tqdm import trange
clrs = ['red','blue','green','cyan']

dirs = np.sort(glob.glob('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/trx_files/'+
                "*"))

imgs=[]
imgs1 = []
for dir_ in dirs:
    fname_mat = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/trx_files/2020-3-16_01_54_23_358257_compressed.avi.predictions.analysis_traces_reassembled_centres/scores_run.mat'
    data =loadmat(fname_mat)
    
    # get length
    temp = data['allScores'][0][0][0][0][0].squeeze()
    length = temp.shape[0]

    # get starts/ends
    t0s = data['allScores'][0][0][5].squeeze()
    #print (t0s.shape)
    t1s = data['allScores'][0][0][6].squeeze()
    
    img = np.zeros((4,length),'float32')
    img1 = np.zeros((4,length),'float32')
    for k in trange(t0s.shape[0]):
    #for k in [0]:
        starts = t0s[k].squeeze()
        ends = t1s[k].squeeze()
        #print (k,starts.shape)
        for p in range(starts.shape[0]):
            img[k,starts[p]:ends[p]]+=k+1
            img1[k,starts[p]:ends[p]]+=1
            
    idx = np.where(img==0)
    img[idx]=np.nan
    imgs.append(img)
    imgs1.append(img1)
    
imgs=np.hstack(imgs)
imgs1=np.hstack(imgs1)


100%|██████████| 4/4 [00:00<00:00, 15.94it/s]


In [18]:
matplotlib.use('Agg')

fig = plt.figure(figsize=(100,5))
ax=plt.subplot(2,1,1)


plt.xticks([])
print (imgs.shape)
names = ['female','male','pup1','pup2']
clrs = ['red','blue','magenta','green']
plt.yticks(np.arange(4),names)
from matplotlib import colors
cmap = colors.ListedColormap(clrs)

plt.imshow(imgs,aspect='auto',interpolation='none',
          cmap=cmap)

ax=plt.subplot(2,1,2)
sums = np.nansum(imgs1, axis= 0)
print (sums.shape)

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    #y_smooth = np.convolve(y, box, mode='same')
    y_smooth = np.convolve(y,np.ones(box_pts,dtype=int),'same')
    
    return y_smooth

window = 60
t = np.arange(imgs.shape[1])/25./window
img2 = np.zeros((4,imgs.shape[1]))
for k in range(4):
    temp = imgs1[k]
    temp = smooth(temp,25*window)    
    #img2[k]=temp
    #temp = smooth(temp,25)    
    
    plt.plot(t,temp,clrs[k])

#plt.imshow(img2,aspect='auto',interpolation='none')#    cmap=cmap)    
plt.xticks(np.arange(0,600,10))
plt.xlim(t[0],t[-1])
plt.xlabel("Time (minutes)")
plt.suptitle("Run behaviors for all animals ("+str(window)+ "sec smoothing window)", fontsize=20)
plt.savefig('/home/cat/fig.png')
#plt.show()



(4, 899880)
(899880,)


In [100]:
temp = data['allScores'][0][0][0][0][k].squeeze()
print(temp.shape)


(89988,)


In [63]:
t0s = data['allScores'][0][0][5].squeeze()
print (t0s.shape)
t1s = data['allScores'][0][0][6].squeeze()

(4,)


In [167]:
fname_mat = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/jaaba/trx_all_21300_21500.mat'
data = loadmat(fname_mat)

In [168]:
print (data['trx'].shape)
print (data['trx'][0][0][1].shape)

data['trx'][0][0][1][0][0] = 44
data['trx'][0][0][1][0] = np.array(data['trx'][0][0][1][0],dtype)

print (data['trx'][0][0][1][0][0])

(1, 4)
(1, 200)
44.0


In [ ]:
  
    
from numpy.core.records import fromarrays
from scipy.io import savemat

names = [
 'x',
 'y',
 'theta',
 'a',
 'b',
 'nframes',
 'firstframe',
 'endframe',
 'off',
 'id',
 'x_mm',
 'y_mm',
 'theta_mm',
 'a_mm',
 'b_mm',
 'sex',
 'dt',
 'fps',
 'timestamps']

data = [
 x,
 y,
 theta,
 a,
 b,
 nframes,
 firstframe,
 endframe,
 off,
 id_,
 x_mm,
 y_mm,
 theta_mm,
 a_mm,
 b_mm,
 sex,
 dt,
 fps,
 timestamps
]


            
#myrec = fromarrays([[1, 10], [2, 20]], names=['field1', 'field2'])
#savemat('p.mat', {'myrec': myrec})

myrec = fromarrays(data, 
                   names=names)
#savemat('p.mat', {'myrec': myrec})


savemat("/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/jaaba/trx.mat", 
        {'myrec':myrec})
    

In [11]:
fnames = glob.glob('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/post_processing/*.npy')

for fname in fnames:
    data = np.load(fname)
    
    data2 = []
    for f in range(0,56,14):
        temp = data[:,f:f+14]
        temp = np.nanmedian(temp,axis=1)
        data2.append(temp)
        
    data2=np.array(data2)
    np.save(fname[:-4]+'_centres.npy',data2)
    print (data2.shape)

(4, 89988, 2)
(4, 89985, 2)
(4, 89987, 2)
(4, 89988, 2)
(4, 89987, 2)
(4, 89987, 2)
(4, 89988, 2)
(4, 89989, 2)
(4, 89976, 2)
(4, 89988, 2)


In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import decomposition
from sklearn import datasets
from sklearn.preprocessing import scale

# load iris dataset
iris = datasets.load_iris()

X = scale(iris.data)
y = iris.target

# apply PCA
pca = decomposition.PCA(n_components=4)
X = pca.fit_transform(X)

loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2', "PC2", "PC3"], index=iris.feature_names)
print (loadings)


                        PC1       PC2       PC2       PC3
sepal length (cm)  0.521066  0.377418 -0.719566 -0.261286
sepal width (cm)  -0.269347  0.923296  0.244382  0.123510
petal length (cm)  0.580413  0.024492  0.142126  0.801449
petal width (cm)   0.564857  0.066942  0.634273 -0.523597


In [18]:
d1 = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed_fixed.npy')
print (d1.shape)

(89989, 4, 14, 2)


In [19]:
# angles = np.zeros((track.tracks.shape[0], 
#                    track.tracks.shape[1], 
#                    'float32'))

angles = np.zeros((d1.shape[0], 
                   d1.shape[1]), 
                   'float32')

deg_scale = 180/3.1415926

for k in trange(10000):
    for a in range(d1.shape[1]):
        x = d1[k,a,:,0]
        y = d1[k,a,:,1]
        idx = np.where(np.isnan(x)==False)[0]
        if idx.shape[0]>0:
            x=x[idx]
            y=y[idx]
            m,b = np.polyfit(x, y, 1)
            #print ("slope :", m, "  intercept: ", b)
            angle = np.arctan(m)*deg_scale
            angles[k,a] = angle
            #print ("agnel: ", angle)      
            

100%|██████████| 10000/10000 [00:06<00:00, 1578.38it/s]


In [ ]:
plt.scatter()

In [87]:
# remove outliars
def reject_outliers(data, m = 20):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    idx = np.where(s<m)[0]
    
    return idx
    #return data[s<m]
        
deg_scale = 180/3.1415926

k_start= 1060
a = 0
ctr=1
for k in range(k_start,k_start+10,1):
    #ax = plt.gca()
    ax =plt.subplot(2,5,ctr)
    x = d1[k,a,5:10,0]
    y = d1[k,a,5:10,1]
    idx = np.where(np.isnan(x)==False)[0]
    if idx.shape[0]>0:
        x=x[idx]
        y=y[idx]
        m,b = np.polyfit(y, x, 1)
        angle = np.arctan(m)*deg_scale #+90
        if x[0]<x[-1]:
            angle+=180
        print ("angle: ", angle) 
        
    plt.title(str(round(angle,1))+" "+str(k))

    locs = np.vstack((x,y)).T
    plt.scatter(locs[:,0],
                locs[:,1],
                c='blue',
                label='original')
    y_vals = np.array(ax.get_ylim())
    x_vals = b + m * y_vals
    plt.plot(x_vals, y_vals, '--')
    plt.xlim(0,1280)
    plt.ylim(1024,0)

    # rotate the thang
    if False:
        theta = np.radians(angle)
        c, s = np.cos(theta), np.sin(theta)
        R = np.array(((c, -s), (s, c)))
        print (locs.shape)
        #locs = R*locs.T
        locs = locs@R



        # Reject outliers that are substantially outside of data
        x = reject_outliers(locs[:,0])
        y = reject_outliers(locs[:,1])

        major = np.max(x)-np.min(x)
        minor = np.max(y)-np.min(y)
        print ("major: ", major, "  minor: ", minor)

        # 
        plt.scatter(locs[:,0], 
                    locs[:,1],
                    c='red',
                   label='rotated')
    
    ctr+=1
plt.legend()
plt.show()

angle:  155.36937322044065
angle:  156.3964094037172
angle:  158.820461411969
angle:  162.98178696290174
angle:  163.0214896740499
angle:  163.20703257794614
angle:  163.1095933974912
angle:  164.4740648932155
angle:  165.45458477460073
angle:  169.03968922275195
